In [1]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [2]:
import kaggle
import os
import shutil
#! cp kaggle.json ~/.kaggle/
#I Received An Error After Running Code Below(I Searched And Found Out It Works Only For Linux And Mac) 
#! mkdir ~/.kaggle
#! chmod 600 ~/.kaggle/kaggle.json

#This Code Is Windows-Friendly 
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True) 
shutil.copy("kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

In [3]:
# I Couldn't Import The Dataset Due To 403 Client Error So I Downloaded Another Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np
import time

In [4]:
df = pd.read_csv("Retail_Transaction_Dataset.csv")
df

,CustomerID,ProductID,Quantity,Price,TransactionDate,PaymentMethod,StoreLocation,ProductCategory,DiscountApplied(%),TotalAmount
0,109318,C,7,80.079844,12/26/2023 12:32,Cash,"176 Andrew Cliffs\nBaileyfort, HI 93354",Books,18.677100,455.862764
1,993229,C,4,75.195229,8/5/2023 0:00,Cash,"11635 William Well Suite 809\nEast Kara, MT 19483",Home Decor,14.121365,258.306546
2,579675,A,8,31.528816,3/11/2024 18:51,Cash,"910 Mendez Ville Suite 909\nPort Lauraland, MO...",Books,15.943701,212.015651
3,799826,D,5,98.880218,10/27/2023 22:00,PayPal,"87522 Sharon Corners Suite 500\nLake Tammy, MO...",Books,6.686337,461.343769
4,121413,A,7,93.188512,12/22/2023 11:38,Cash,"0070 Michelle Island Suite 143\nHoland, VA 80142",Electronics,4.030096,626.030484
...,...,...,...,...,...,...,...,...,...,...
99995,726461,A,2,56.078258,7/17/2023 16:59,Credit Card,"3632 Darren Station Apt. 553\nEricaborough, RI...",Clothing,18.345145,91.581240
99996,328056,A,6,88.516406,5/30/2023 9:04,Credit Card,"821 Taylor Shoals\nEvansville, IL 70845",Electronics,3.995541,509.878179
99997,887304,B,4,72.385564,8/25/2023 7:59,Credit Card,"50653 Kara Lakes\nStephanieborough, RI 94492",Clothing,17.423979,239.092472
99998,326401,C,5,66.542239,2/5/2024 19:45,PayPal,"18756 Mcfarland Way Suite 866\nBarnettside, PR...",Electronics,14.345018,284.983717


In [5]:
df.info() #There isn't Any Nulls In This Dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CustomerID          100000 non-null  int64  
 1   ProductID           100000 non-null  object 
 2   Quantity            100000 non-null  int64  
 3   Price               100000 non-null  float64
 4   TransactionDate     100000 non-null  object 
 5   PaymentMethod       100000 non-null  object 
 6   StoreLocation       100000 non-null  object 
 7   ProductCategory     100000 non-null  object 
 8   DiscountApplied(%)  100000 non-null  float64
 9   TotalAmount         100000 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 7.6+ MB


In [6]:
df.describe() #I have No Wrong Data/Outlier

,CustomerID,Quantity,Price,DiscountApplied(%),TotalAmount
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,500463.982180,5.009290,55.067344,10.020155,248.334955
std,288460.917524,2.579808,25.971567,5.779534,184.554792
min,14.000000,1.000000,10.000430,0.000046,8.274825
25%,250693.750000,3.000000,32.549474,5.001013,95.163418
50%,499679.000000,5.000000,55.116789,10.030353,200.368393
75%,751104.750000,7.000000,77.456763,15.018367,362.009980
max,999997.000000,9.000000,99.999284,19.999585,896.141242


In [7]:
df.isnull().sum() #Double Checking Number Of Nulls

CustomerID            0
ProductID             0
Quantity              0
Price                 0
TransactionDate       0
PaymentMethod         0
StoreLocation         0
ProductCategory       0
DiscountApplied(%)    0
TotalAmount           0
dtype: int64

In [8]:
print(df["ProductCategory"].value_counts()) #Checking Which Column Is More Valueable To Me
print(df["ProductID"].value_counts())

ProductCategory
Clothing       25056
Books          25031
Electronics    24990
Home Decor     24923
Name: count, dtype: int64
ProductID
C    25209
D    25042
B    24997
A    24752
Name: count, dtype: int64


In [9]:
df["PaymentMethod"].value_counts()

PaymentMethod
PayPal         25067
Credit Card    25034
Cash           25008
Debit Card     24891
Name: count, dtype: int64

In [10]:
#Extracting Months From df["TransactionDate"]
df["TransactionDate"] = pd.to_datetime(df["TransactionDate"])
df["TransActionMonth"] = df["TransactionDate"].dt.month
print(df["TransActionMonth"].value_counts())
df["TransActionMonth"]

TransActionMonth
7     8597
1     8543
8     8498
12    8473
3     8457
5     8388
10    8325
11    8298
6     8243
9     8181
2     8076
4     7921
Name: count, dtype: int64


0        12
1         8
2         3
3        10
4        12
         ..
99995     7
99996     5
99997     8
99998     2
99999     2
Name: TransActionMonth, Length: 100000, dtype: int32

In [11]:
#Reducing df["StoreLocation"] Into One Word
df["StoreLocation"] = df["StoreLocation"].str.replace(" ", "") #Question 1: Should I remove ","s In Addresses?
df["StoreLocation"]

0                   176AndrewCliffs\nBaileyfort,HI93354
1            11635WilliamWellSuite809\nEastKara,MT19483
2         910MendezVilleSuite909\nPortLauraland,MO99563
3         87522SharonCornersSuite500\nLakeTammy,MO76234
4            0070MichelleIslandSuite143\nHoland,VA80142
                              ...                      
99995    3632DarrenStationApt.553\nEricaborough,RI37695
99996               821TaylorShoals\nEvansville,IL70845
99997          50653KaraLakes\nStephanieborough,RI94492
99998    18756McfarlandWaySuite866\nBarnettside,PR50529
99999                   8046HullDrive\nPaulstad,GU87218
Name: StoreLocation, Length: 100000, dtype: object

In [12]:
#Changing df["ProductCategory"] And df["PaymentMethod"] Into Interger Values
le = LabelEncoder()
df["ProductCategoryEncoded"] = le.fit_transform(df["ProductCategory"])
print(df["ProductCategoryEncoded"])
df["PaymentMethodEncoded"] = le.fit_transform(df["PaymentMethod"])
print(df["PaymentMethodEncoded"])

0        0
1        3
2        0
3        0
4        2
        ..
99995    1
99996    2
99997    1
99998    2
99999    3
Name: ProductCategoryEncoded, Length: 100000, dtype: int32
0        0
1        0
2        0
3        3
4        0
        ..
99995    1
99996    1
99997    1
99998    3
99999    2
Name: PaymentMethodEncoded, Length: 100000, dtype: int32


In [13]:
#Unifying Data
df
for col in ["Price", "Quantity", "StoreLocation", "ProductCategoryEncoded", "TransActionMonth", "PaymentMethodEncoded", "DiscountApplied(%)", "TotalAmount"]:
    df[col] = df[col].astype(str)
    
df["UnifiedFeatures"] = df["Price"] + " " + df["Quantity"] + " " + df["StoreLocation"] + " " + df["ProductCategoryEncoded"] + " " + df["TransActionMonth"] + " " + df["PaymentMethodEncoded"] + " " + df["DiscountApplied(%)"] + " " + df["TotalAmount"]

In [14]:
df["UnifiedFeatures"]

0        80.07984415 7 176AndrewCliffs\nBaileyfort,HI93...
1        75.19522942 4 11635WilliamWellSuite809\nEastKa...
2        31.52881648 8 910MendezVilleSuite909\nPortLaur...
3        98.88021828 5 87522SharonCornersSuite500\nLake...
4        93.18851246 7 0070MichelleIslandSuite143\nHola...
                               ...                        
99995    56.07825789 2 3632DarrenStationApt.553\nEricab...
99996    88.51640554 6 821TaylorShoals\nEvansville,IL70...
99997    72.38556379 4 50653KaraLakes\nStephanieborough...
99998    66.54223948 5 18756McfarlandWaySuite866\nBarne...
99999    38.08776628 5 8046HullDrive\nPaulstad,GU87218 ...
Name: UnifiedFeatures, Length: 100000, dtype: object

In [15]:
#Putting All Values Into One vector
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(df["UnifiedFeatures"])
vector

<100000x536163 sparse matrix of type '<class 'numpy.int64'>'
	with 887306 stored elements in Compressed Sparse Row format>

In [16]:
#similarities = cosine_similarity(vector)

#similarities #This Operation Couldn't Be Done Because Of Its Volume(MemoryError)

In [17]:
#Finding Nearest Neighbors To Each Customer's Transaction
start_time = time.time()
nn = NearestNeighbors(n_neighbors=6, metric='cosine') 
nn.fit(vector)
suggestion = nn.kneighbors(vector, return_distance=False)
print(suggestion)
end_time = time.time()
duration = end_time - start_time
print(duration)

[[    0 49084 35220 33537 59315 95596]
 [    1 47627 55063 14361 60277 27571]
 [    2 64053 61813 69940 79133 73013]
 ...
 [99997 98018 11195  6009 61119  7810]
 [99998 32051  9693  5860 39272 90755]
 [99999 11817 42850 76328 28616 80233]]
155.58349347114563


In [37]:
#Finding Best Match To A Customer
customer = int(input("Enter your Customer ID:"))
try:
    customer_index =  df[df["CustomerID"] == customer].index[0]
    customer_df = np.c_[df["CustomerID"], suggestion]
    np.argsort(suggestion[customer_index])
    top_indices = suggestion[customer_index]
    print(f"Top 5 suggestions for customer {customer}:")
    for i in top_indices[1:6]: 
        print(df.loc[i , ["ProductID", "Price", "ProductCategory", "DiscountApplied(%)"]])

except:
    print("Customer ID Not Found.")

Enter your Customer ID: 121413


Top 5 suggestions for customer 121413:
ProductID                       D
Price                 93.06268665
ProductCategory          Clothing
DiscountApplied(%)    12.07834507
Name: 65551, dtype: object
ProductID                       B
Price                 93.21287982
ProductCategory       Electronics
DiscountApplied(%)    0.129943597
Name: 23932, dtype: object
ProductID                       B
Price                 93.24369373
ProductCategory        Home Decor
DiscountApplied(%)    12.21412892
Name: 46487, dtype: object
ProductID                       C
Price                    13.36108
ProductCategory             Books
DiscountApplied(%)    12.07196599
Name: 87373, dtype: object
ProductID                       D
Price                 93.17155697
ProductCategory        Home Decor
DiscountApplied(%)    12.95009525
Name: 61873, dtype: object
